<a href="https://colab.research.google.com/github/jss265/595R---Deep-Learning-for-Engineers/blob/main/HW%201%20-%20MLP/HW1_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
print('Hello Woldies')
print('Hello again!...')
print('test from colab')

Hello Woldies
Hello again!...
test from colab
